In [1]:
from api.BinanceData import BinanceData
from utils.CleanData import CleanData
from variables.QuantitativeFunc import QuantitativeFunc
from variables.Target import Target
from variables.TradingIndicators import TradingIndicators
import pandas as pd

In [2]:
# Import de données
BinanceData = BinanceData()
data = BinanceData.load_data("BTCUSDT", days=30)
print(data.head())

       open_time             open             high              low  \
0  1756042800000  114714.17000000  114714.18000000  114576.43000000   
1  1756043100000  114643.19000000  114781.21000000  114625.84000000   
2  1756043400000  114734.66000000  114734.66000000  114625.96000000   
3  1756043700000  114625.96000000  114625.97000000  114555.00000000   
4  1756044000000  114555.00000000  114564.69000000  114382.64000000   

             close        volume     close_time quote_asset_volume  \
0  114643.19000000   40.94075000  1756043099999   4693000.29881940   
1  114734.65000000  125.02312000  1756043399999  14344176.35065610   
2  114625.97000000   37.79336000  1756043699999   4334522.03905600   
3  114555.00000000   53.41280000  1756043999999   6120598.54896650   
4  114459.10000000  139.15520000  1756044299999  15934617.19877640   

   number_of_trades taker_buy_base   taker_buy_quote ignore  
0              8468    13.82225000  1584358.29425670      0  
1             12120    44.36

In [3]:
# Nettoyage des données 
cleaner = CleanData()
cleaned_data = cleaner.clean_klines_data(data)
print(cleaned_data.index.name)
print(cleaned_data.head())

open_time
                          open       high        low      close     volume  \
open_time                                                                    
2025-08-24 13:40:00  114714.17  114714.18  114576.43  114643.19   40.94075   
2025-08-24 13:45:00  114643.19  114781.21  114625.84  114734.65  125.02312   
2025-08-24 13:50:00  114734.66  114734.66  114625.96  114625.97   37.79336   
2025-08-24 13:55:00  114625.96  114625.97  114555.00  114555.00   53.41280   
2025-08-24 14:00:00  114555.00  114564.69  114382.64  114459.10  139.15520   

                                 close_time  quote_asset_volume  \
open_time                                                         
2025-08-24 13:40:00 2025-08-24 13:44:59.999        4.693000e+06   
2025-08-24 13:45:00 2025-08-24 13:49:59.999        1.434418e+07   
2025-08-24 13:50:00 2025-08-24 13:54:59.999        4.334522e+06   
2025-08-24 13:55:00 2025-08-24 13:59:59.999        6.120599e+06   
2025-08-24 14:00:00 2025-08-24 14:04:59.9

In [4]:
# Export des données pour travailler sur un ficheir Excel plutôt que de requêter l'API Binance à chaque fois
cleaned_data.to_excel("../data/données_test.xlsx", index=False)

In [5]:
# Import des données depuis le fichier Excel
data = pd.read_excel("../data/données_test.xlsx")

# Création des variables rendement et la cible 
data["return"] = QuantitativeFunc.return_(data['close'])
data["return_10"] = QuantitativeFunc.return_10(data['close'])
data["target"] = Target.compute(data["return"], threshold=0.0005)

# ajout des indicateurs techniques
data = TradingIndicators.add_sma(data, price_col="close", window=20, new_col="SMA_20")
data = TradingIndicators.add_ema(data, price_col="close", window=12, new_col="EMA_12")
data = TradingIndicators.add_macd(data, price_col="close")
data = TradingIndicators.add_bollinger_bands(data, price_col="close")
data = TradingIndicators.add_rsi(data, price_col="close")
data = TradingIndicators.add_atr(data, high_col="high", low_col="low", close_col="close", window=14, new_col="ATR_14")
data = TradingIndicators.add_high_low_range(data, high_col="high", low_col="low")
data = TradingIndicators.add_buy_pressure(data, high_col="high", low_col="low", close_col="close")
data = TradingIndicators.add_realized_volatility(data, returns_col="return", window=14)

print(data.head())
print(data["target"].value_counts())
print(data.columns)

        open       high        low      close     volume  \
0  114714.17  114714.18  114576.43  114643.19   40.94075   
1  114643.19  114781.21  114625.84  114734.65  125.02312   
2  114734.66  114734.66  114625.96  114625.97   37.79336   
3  114625.96  114625.97  114555.00  114555.00   53.41280   
4  114555.00  114564.69  114382.64  114459.10  139.15520   

               close_time  quote_asset_volume  number_of_trades  \
0 2025-08-24 13:44:59.999        4.693000e+06              8468   
1 2025-08-24 13:49:59.999        1.434418e+07             12120   
2 2025-08-24 13:54:59.999        4.334522e+06              6747   
3 2025-08-24 13:59:59.999        6.120599e+06              5271   
4 2025-08-24 14:04:59.999        1.593462e+07             16773   

   taker_buy_quote    return  ...       MACD  MACD_Signal  Bollinger_SMA  \
0     1.584358e+06       NaN  ...   0.000000     0.000000            NaN   
1     5.090518e+06  0.000797  ...   7.295954     1.459191            NaN   
2     5.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Prix de clôture (noir, trait plein)
plt.plot(data["open_time"], data["close"], label="Close", color="black", linestyle="-", linewidth=1)

# Moyennes mobiles (pointillés)
plt.plot(data["open_time"], data["SMA_20"], label="SMA 20", color="blue", linestyle="--", linewidth=1)
plt.plot(data["open_time"], data["EMA_12"], label="EMA 12", color="orange", linestyle="--", linewidth=1)

# Bandes de Bollinger (pointillés verts)
plt.plot(data["open_time"], data["Bollinger_Upper"], label="Bollinger Upper", color="green", linestyle=":", linewidth=1)
plt.plot(data["open_time"], data["Bollinger_Lower"], label="Bollinger Lower", color="green", linestyle=":", linewidth=1)

# Remplir l'espace entre les bandes (optionnel)
plt.fill_between(data["open_time"], data["Bollinger_Lower"], data["Bollinger_Upper"], color="green", alpha=0.1)

# Légende et grille
plt.title("Prix avec SMA, EMA et Bollinger Bands (5min)")
plt.xlabel("Temps")
plt.ylabel("Prix")
plt.legend()
plt.grid(True)

plt.show()